In [4]:
import pandas as pd

# Load datasets
customer_profile = pd.read_csv('/kaggle/input/robi-datathon-3-0/TBL_CUSTOMER_PROFILE.csv')
product_catalogue = pd.read_csv('/kaggle/input/robi-datathon-3-0/TBL_PRODUCT_CATALOGUE.csv')
pack_purchase = pd.read_csv('/kaggle/input/robi-datathon-3-0/TBL_PACK_PURCHASE.csv')
data_consumption = pd.read_csv('/kaggle/input/robi-datathon-3-0/TBL_DATA_CONSUMPTION.csv')
q1_customers = pd.read_csv('/kaggle/input/datathon3-0-question/Q1-customers.csv')


In [5]:
print(customer_profile.shape)
print(product_catalogue.shape)
print(pack_purchase.shape)
print(data_consumption.shape)
print(q1_customers.shape)


(342643, 14)
(2049, 7)
(1208710, 4)
(23375373, 4)
(17843, 1)


In [6]:
# Merge customer profiles with pack purchases and product catalog
pack_purchase_merged = pd.merge(pack_purchase, product_catalogue, on='PACK_ID', how='left')
customer_data = pd.merge(customer_profile, pack_purchase_merged, on='CUSTOMER_ID', how='left')

# Handle missing values, if any
customer_data.fillna(0, inplace=True)
print(pack_purchase_merged.shape)
print(customer_data.shape)


(1208710, 10)
(1208718, 23)


In [7]:
# Convert FIRST_CONNECTION_DATE to datetime
customer_data['FIRST_CONNECTION_DATE'] = pd.to_datetime(customer_data['FIRST_CONNECTION_DATE'])
customer_data['EVENT_DATE'] = pd.to_datetime(customer_data['EVENT_DATE'])

# Extract features like days since first connection
customer_data['DAYS_SINCE_FIRST_CONNECTION'] = (customer_data['EVENT_DATE'] - customer_data['FIRST_CONNECTION_DATE']).dt.days

# Aggregate data consumption to get total consumption per customer
data_consumption_agg = data_consumption.groupby('CUSTOMER_ID').agg({
    'TOTAL_VOLUME_GB': 'sum',
    'VOLUME_GB_IN_HIGH_BANDWIDTH': 'sum'
}).reset_index()

# Merge aggregated data consumption with customer data
customer_data = pd.merge(customer_data, data_consumption_agg, on='CUSTOMER_ID', how='left')

# Fill missing values again if any
customer_data.fillna(0, inplace=True)

# Filter only relevant features for model training
features = [
    'SERVICE_TYPE_x', 'CONNECTION_TYPE', 'ROUTER_CATEGORY', 
    'STREAMING_VOL_PERCENTAGE', 'WEB_BROWSING_VOL_PERCENTAGE', 
    'IM_VOICE_CALL_VOL_PERCENTAGE', 'SNS_VOL_PERCENTAGE', 
    'FILE_ACCESS_VOL_PERCENTAGE', 'GAME_VOL_PERCENTAGE', 
    'NAVIGATION_VOL_PERCENTAGE', 'EMAIL_VOL_PERCENTAGE', 
    'OTHERS_VOL_PERCENTAGE', 'DAYS_SINCE_FIRST_CONNECTION', 
    'TOTAL_VOLUME_GB', 'VOLUME_GB_IN_HIGH_BANDWIDTH'
]

X = customer_data[features]
y = customer_data['PACK_ID']


In [8]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [9]:
from sklearn.preprocessing import LabelEncoder

# Create a LabelEncoder instance
label_encoder = LabelEncoder()

# Apply LabelEncoder to each categorical column
categorical_columns = ['SERVICE_TYPE_x', 'CONNECTION_TYPE', 'ROUTER_CATEGORY']

for column in categorical_columns:
    X[column] = label_encoder.fit_transform(X[column])

print(X.shape)
display(X.head())


/tmp/ipykernel_33/1696699246.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])
/tmp/ipykernel_33/1696699246.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])


(1208718, 15)


/tmp/ipykernel_33/1696699246.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X[column] = label_encoder.fit_transform(X[column])


,SERVICE_TYPE_x,CONNECTION_TYPE,ROUTER_CATEGORY,STREAMING_VOL_PERCENTAGE,WEB_BROWSING_VOL_PERCENTAGE,IM_VOICE_CALL_VOL_PERCENTAGE,SNS_VOL_PERCENTAGE,FILE_ACCESS_VOL_PERCENTAGE,GAME_VOL_PERCENTAGE,NAVIGATION_VOL_PERCENTAGE,EMAIL_VOL_PERCENTAGE,OTHERS_VOL_PERCENTAGE,DAYS_SINCE_FIRST_CONNECTION,TOTAL_VOLUME_GB,VOLUME_GB_IN_HIGH_BANDWIDTH
0,1,1,1,70.0205,14.6609,3.6914,8.5530,2.1653,0.0297,0.0376,0.0,0.8416,-1,987.1017,984.4302
1,1,1,1,70.0205,14.6609,3.6914,8.5530,2.1653,0.0297,0.0376,0.0,0.8416,-1,987.1017,984.4302
2,1,1,1,70.0205,14.6609,3.6914,8.5530,2.1653,0.0297,0.0376,0.0,0.8416,-1,987.1017,984.4302
3,1,1,1,70.0205,14.6609,3.6914,8.5530,2.1653,0.0297,0.0376,0.0,0.8416,-1,987.1017,984.4302
4,1,0,0,72.1494,11.6056,3.2767,1.6749,10.2768,0.0000,0.2096,0.0,0.8069,-1,2.1406,1.9881


In [10]:
# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [11]:
print(X_train.shape)
print(X_test.shape)

(966974, 15)
(241744, 15)


In [12]:
# Train a RandomForest model
model = RandomForestClassifier(n_estimators=2, random_state=42)
model.fit(X_train, y_train)

# Evaluate the model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Model Accuracy: {accuracy}')

Model Accuracy: 0.29180869018465816


In [13]:
# Prepare Q1-customers data similar to training data
q1_customers_data = pd.merge(q1_customers, customer_profile, on='CUSTOMER_ID', how='left')
q1_customers_data = pd.merge(q1_customers_data, data_consumption_agg, on='CUSTOMER_ID', how='left')

q1_customers_data['FIRST_CONNECTION_DATE'] = pd.to_datetime(q1_customers_data['FIRST_CONNECTION_DATE'])
q1_customers_data['DAYS_SINCE_FIRST_CONNECTION'] = (pd.Timestamp('now') - q1_customers_data['FIRST_CONNECTION_DATE']).dt.days
q1_customers_data.fillna(0, inplace=True)

q1_customers_data.head()

,CUSTOMER_ID,SERVICE_TYPE,FIRST_CONNECTION_DATE,CONNECTION_TYPE,ROUTER_CATEGORY,STREAMING_VOL_PERCENTAGE,WEB_BROWSING_VOL_PERCENTAGE,IM_VOICE_CALL_VOL_PERCENTAGE,SNS_VOL_PERCENTAGE,FILE_ACCESS_VOL_PERCENTAGE,GAME_VOL_PERCENTAGE,NAVIGATION_VOL_PERCENTAGE,EMAIL_VOL_PERCENTAGE,OTHERS_VOL_PERCENTAGE,TOTAL_VOLUME_GB,VOLUME_GB_IN_HIGH_BANDWIDTH,DAYS_SINCE_FIRST_CONNECTION
0,CD1F7B49575998,Standard,1970-01-01 00:00:00.020230327,Cable Broadband,High End,70.9129,8.6307,7.3217,8.8879,3.9617,0.0100,0.0043,0.0000,0.2708,858.0553,854.2474,19867
1,CD1F7B86F083B7,Standard,1970-01-01 00:00:00.020230508,Fixed Wireless,Low End,67.7617,15.4705,3.6035,10.8574,1.0199,0.0054,0.0383,0.0000,1.2432,798.1641,797.0118,19867
2,CD1F7C8C57DA02,Standard,1970-01-01 00:00:00.020230320,Fixed Wireless,Low End,57.8069,21.9196,2.1142,15.5177,2.2513,0.0369,0.1074,0.0004,0.2456,322.5204,322.0202,19867
3,CD1F7DA21C1ED4,Standard,1970-01-01 00:00:00.020230311,Fiber Optic,Low End,49.6374,34.8527,3.6523,1.1167,7.9447,0.0077,0.0190,0.0000,2.7695,748.2582,724.2775,19867
4,CD1F7EBFC7032A,Standard,1970-01-01 00:00:00.020230511,Fiber Optic,High End,49.0021,22.3278,16.7901,5.9855,3.2701,0.1481,0.0043,0.0000,2.4720,1140.7191,1131.3555,19867


In [14]:
q1_customers_data.rename(columns={'SERVICE_TYPE': 'SERVICE_TYPE_x'}, inplace=True)
q1_customers_data.head()

,CUSTOMER_ID,SERVICE_TYPE_x,FIRST_CONNECTION_DATE,CONNECTION_TYPE,ROUTER_CATEGORY,STREAMING_VOL_PERCENTAGE,WEB_BROWSING_VOL_PERCENTAGE,IM_VOICE_CALL_VOL_PERCENTAGE,SNS_VOL_PERCENTAGE,FILE_ACCESS_VOL_PERCENTAGE,GAME_VOL_PERCENTAGE,NAVIGATION_VOL_PERCENTAGE,EMAIL_VOL_PERCENTAGE,OTHERS_VOL_PERCENTAGE,TOTAL_VOLUME_GB,VOLUME_GB_IN_HIGH_BANDWIDTH,DAYS_SINCE_FIRST_CONNECTION
0,CD1F7B49575998,Standard,1970-01-01 00:00:00.020230327,Cable Broadband,High End,70.9129,8.6307,7.3217,8.8879,3.9617,0.0100,0.0043,0.0000,0.2708,858.0553,854.2474,19867
1,CD1F7B86F083B7,Standard,1970-01-01 00:00:00.020230508,Fixed Wireless,Low End,67.7617,15.4705,3.6035,10.8574,1.0199,0.0054,0.0383,0.0000,1.2432,798.1641,797.0118,19867
2,CD1F7C8C57DA02,Standard,1970-01-01 00:00:00.020230320,Fixed Wireless,Low End,57.8069,21.9196,2.1142,15.5177,2.2513,0.0369,0.1074,0.0004,0.2456,322.5204,322.0202,19867
3,CD1F7DA21C1ED4,Standard,1970-01-01 00:00:00.020230311,Fiber Optic,Low End,49.6374,34.8527,3.6523,1.1167,7.9447,0.0077,0.0190,0.0000,2.7695,748.2582,724.2775,19867
4,CD1F7EBFC7032A,Standard,1970-01-01 00:00:00.020230511,Fiber Optic,High End,49.0021,22.3278,16.7901,5.9855,3.2701,0.1481,0.0043,0.0000,2.4720,1140.7191,1131.3555,19867


In [15]:

# Predict which new products customers are likely to purchase
X_q1 = q1_customers_data[features]

for column in categorical_columns:
    X_q1[column] = label_encoder.fit_transform(X_q1[column])
    
q1_predictions = model.predict(X_q1)

# If no purchase is likely, set prediction to -1
q1_customers_data['PREDICTED_PACK_ID'] = q1_predictions
q1_customers_data.loc[q1_customers_data['PREDICTED_PACK_ID'] == 0, 'PREDICTED_PACK_ID'] = -1

# Save predictions
q1_customers_data[['CUSTOMER_ID', 'PREDICTED_PACK_ID']].to_csv('q1_predictions.csv', index=False)
q1_customers_data.head()

/tmp/ipykernel_33/3070458174.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_q1[column] = label_encoder.fit_transform(X_q1[column])
/tmp/ipykernel_33/3070458174.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_q1[column] = label_encoder.fit_transform(X_q1[column])
/tmp/ipykernel_33/3070458174.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas

,CUSTOMER_ID,SERVICE_TYPE_x,FIRST_CONNECTION_DATE,CONNECTION_TYPE,ROUTER_CATEGORY,STREAMING_VOL_PERCENTAGE,WEB_BROWSING_VOL_PERCENTAGE,IM_VOICE_CALL_VOL_PERCENTAGE,SNS_VOL_PERCENTAGE,FILE_ACCESS_VOL_PERCENTAGE,GAME_VOL_PERCENTAGE,NAVIGATION_VOL_PERCENTAGE,EMAIL_VOL_PERCENTAGE,OTHERS_VOL_PERCENTAGE,TOTAL_VOLUME_GB,VOLUME_GB_IN_HIGH_BANDWIDTH,DAYS_SINCE_FIRST_CONNECTION,PREDICTED_PACK_ID
0,CD1F7B49575998,Standard,1970-01-01 00:00:00.020230327,Cable Broadband,High End,70.9129,8.6307,7.3217,8.8879,3.9617,0.0100,0.0043,0.0000,0.2708,858.0553,854.2474,19867,924.0
1,CD1F7B86F083B7,Standard,1970-01-01 00:00:00.020230508,Fixed Wireless,Low End,67.7617,15.4705,3.6035,10.8574,1.0199,0.0054,0.0383,0.0000,1.2432,798.1641,797.0118,19867,1839.0
2,CD1F7C8C57DA02,Standard,1970-01-01 00:00:00.020230320,Fixed Wireless,Low End,57.8069,21.9196,2.1142,15.5177,2.2513,0.0369,0.1074,0.0004,0.2456,322.5204,322.0202,19867,1912.0
3,CD1F7DA21C1ED4,Standard,1970-01-01 00:00:00.020230311,Fiber Optic,Low End,49.6374,34.8527,3.6523,1.1167,7.9447,0.0077,0.0190,0.0000,2.7695,748.2582,724.2775,19867,1606.0
4,CD1F7EBFC7032A,Standard,1970-01-01 00:00:00.020230511,Fiber Optic,High End,49.0021,22.3278,16.7901,5.9855,3.2701,0.1481,0.0043,0.0000,2.4720,1140.7191,1131.3555,19867,256.0
